In [4]:
import random
import tkinter as tk
from tkinter import messagebox

def generate_grid(words, size):
    grid = [['' for _ in range(size)] for _ in range(size)]
    directions = [(0, 1), (1, 0), (1, 1), (-1, 1)]  # Possible directions: right, down, diagonal-down-right, diagonal-up-right
    for word in words:
        direction = random.choice(directions)  # Choose a random direction
        word_length = len(word)
        start_x = random.randint(0, size - 1)  # Start at a random position within the grid
        start_y = random.randint(0, size - 1)
        while not (0 <= start_x + (word_length - 1) * direction[0] < size and 0 <= start_y + (word_length - 1) * direction[1] < size):
            start_x = random.randint(0, size - 1)  # Regenerate start position if word doesn't fit
            start_y = random.randint(0, size - 1)
        for i, letter in enumerate(word):
            x = start_x + i * direction[0]
            y = start_y + i * direction[1]
            grid[x][y] = letter
    # Fill the rest of the grid with random letters
    for i in range(size):
        for j in range(size):
            if grid[i][j] == '':
                grid[i][j] = random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    return grid

# Function to update the grid display
def update_grid_display():
    for i in range(size):
        for j in range(size):
            cell = grid[i][j]
            if (i, j) in clicked_paths:
                labels[i][j].config(bg="blue", fg="white")
            elif (i, j) in ai_solved_paths:
                labels[i][j].config(bg="green", fg="white")
            else:
                labels[i][j].config(bg="white", fg="black")

# Function to handle click events on the grid
def on_cell_click(x, y):
    clicked_paths.append((x, y))
    update_grid_display()
    check_word_start()

# Function to check if the clicked letters form a word
def check_word_start():
    clicked_word = ''.join([grid[i][j] for i, j in clicked_paths])
    for word in words:
        if word.startswith(clicked_word):
            return
    clicked_paths.clear()
    update_grid_display()

# Function to solve the puzzle using backtracking
def solve_puzzle(grid, words):
    def is_valid(x, y):
        return 0 <= x < len(grid) and 0 <= y < len(grid[0])

    def backtrack(x, y, path):
        if len(path) == len(word):
            return path
        for dx, dy in [(0, 1), (1, 0), (1, 1), (-1, 1)]:
            new_x, new_y = x + dx, y + dy
            if is_valid(new_x, new_y) and (new_x, new_y) not in path and grid[new_x][new_y] == word[len(path)]:
                new_path = path + [(new_x, new_y)]
                result = backtrack(new_x, new_y, new_path)
                if result:
                    return result
        return None

    ai_solved_paths = {}
    for word in words:
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == word[0]:
                    path = [(i, j)]
                    result = backtrack(i, j, path)
                    if result:
                        ai_solved_paths[word] = result
                        break
            if word in ai_solved_paths:
                break
    return ai_solved_paths

# Function to handle AI solve button click
def ai_solve():
    global ai_solved_paths, all_words_solved
    ai_solved_paths = solve_puzzle(grid, words)
    all_words_solved = True
    for word, path in ai_solved_paths.items():
        for x, y in path:
            clicked_paths.append((x, y))
    update_grid_display()

# Main function
if __name__ == "__main__":
    words = ["ADARSH", "SANSKAR","SHREYAS"]
    size = 10
    grid = generate_grid(words, size)

    root = tk.Tk()
    root.title("Word Puzzle Game")

    clicked_paths = []
    ai_solved_paths = {}
    labels = [[None for _ in range(size)] for _ in range(size)]

    for i in range(size):
        for j in range(size):
            labels[i][j] = tk.Label(root, text=grid[i][j], width=2, height=1, relief="solid")
            labels[i][j].grid(row=i, column=j)
            labels[i][j].bind("<Button-1>", lambda e, x=i, y=j: on_cell_click(x, y))

    ai_button = tk.Button(root, text="AI Solve", command=ai_solve)
    ai_button.grid(row=size, columnspan=size, sticky="ew")

    root.mainloop()